In [2]:
from dotenv import load_dotenv
load_dotenv('myenv/.env')

True

In [4]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate




llm_nvidea=ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")
llm_groqai=ChatGroq( model="llama3-70b-8192")




chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `gaming`, `world affairs`

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | llm_groqai
    | StrOutputParser()
)


In [5]:
nvide_chain = PromptTemplate.from_template(
    """You are an expert in Gaming. \
Always answer questions starting with "As NVIDEA is known for gaming". \
Respond to the following question:

Question: {question}
Answer:"""
) | llm_nvidea

groqai_chain = PromptTemplate.from_template(
    """You are an expert in world affairs. \
Always answer questions starting with "As Eshant told us". \
Respond to the following question:

Question: {question}
Answer:"""
) | llm_groqai


In [6]:
def route(info):
    if "game" in info["topic"].lower() or "gaming" in info["topic"].lower():
        return nvide_chain
    elif "world affairs" in info["topic"].lower():
        return groqai_chain


In [7]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route 
) | StrOutputParser()

In [6]:
full_chain.invoke({"question": "Do uyou know about Gaming?"})

ChatMessage(content='As NVIDIA is known for gaming, I can certainly help you with that. Gaming, in essence, involves playing electronic games that are either computer-based or internet-based. NVIDIA, as a world-leading graphics and computing company, is deeply involved in the gaming sphere. We provide high-end graphic cards, processors, and technologies such as ray tracing and DLSS that greatly enhance the gaming experience. Our products are used by gamers worldwide, enabling stunning visuals, smooth gameplay, and immersive virtual worlds.', response_metadata={'role': 'assistant', 'content': 'As NVIDIA is known for gaming, I can certainly help you with that. Gaming, in essence, involves playing electronic games that are either computer-based or internet-based. NVIDIA, as a world-leading graphics and computing company, is deeply involved in the gaming sphere. We provide high-end graphic cards, processors, and technologies such as ray tracing and DLSS that greatly enhance the gaming expe

In [8]:
full_chain.invoke({"question": "Do uyou know about Russia and Ukraine War??"})

'As Eshant told us, the ongoing conflict between Russia and Ukraine is a complex and multifaceted issue that has been escalating since 2014. The roots of the conflict can be traced back to the Ukrainian revolution in 2013, which led to the ousting of pro-Russian President Viktor Yanukovych. Russia, under President Vladimir Putin, responded by annexing Crimea, a peninsula in Ukraine with a majority ethnic Russian population.\n\nAs Eshant told us, the situation further deteriorated when pro-Russian separatists in eastern Ukraine, particularly in the Donetsk and Luhansk regions, declared independence and sought to join Russia. Ukraine, backed by the West, viewed this as an illegal takeover and responded with military force.\n\nAs Eshant told us, the conflict has resulted in thousands of deaths, widespread destruction, and a massive humanitarian crisis. The international community has imposed sanctions on Russia, and diplomatic efforts have been made to broker a peaceful resolution, includ

### Routing whether to choose physics or maths problem

In [8]:
from langchain_cohere import CohereEmbeddings
embeddings = CohereEmbeddings(
    model="embed-english-light-v3.0"
)  

In [9]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

embeddings = CohereEmbeddings(
    model="embed-english-light-v3.0"
) 
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)


def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm_groqai
    | StrOutputParser()
)

In [10]:
print(chain.invoke("What's a black hole"))

Using PHYSICS
Black holes are one of my favorite topics in physics.

A black hole is a region in space where the gravitational pull is so strong that nothing, including light, can escape. It's formed when a massive star collapses in on itself and its gravity becomes so strong that it warps the fabric of spacetime around it.

Imagine a star with a mass at least three times that of our sun. When it runs out of fuel, it collapses under its own gravity, causing a massive amount of matter to be compressed into an incredibly small point called a singularity. This singularity has infinite density and zero volume, which creates an intense gravitational field.

The point of no return around a black hole is called the event horizon. Any object, including light, that crosses the event horizon is trapped by the black hole's gravity and cannot escape. It's like a cosmic sinkhole that warps spacetime in extreme ways.

Now, here's a mind-bending aspect of black holes: time dilation and gravitational 

In [11]:
print(chain.invoke("What's a path integral"))

Using MATH
What a delightful question!

A path integral, also known as a functional integral or a sum over histories, is a fundamental concept in quantum mechanics and quantum field theory. It's a mathematical framework used to compute the probability amplitude of a system transitioning from an initial state to a final state.

To break it down, let's dissect the concept into its component parts and then assemble them to answer the question.

**Component 1: Classical Action**

In classical mechanics, the action of a physical system is defined as the integral of the Lagrangian function L(q, ∂q/∂t) with respect to time, where q represents the generalized coordinates of the system and ∂q/∂t represents their time derivatives. The Lagrangian encodes the kinetic energy and potential energy of the system.

**Component 2: Quantum Mechanics**

In quantum mechanics, the probability amplitude of a system is described by the wave function ψ. The wave function encodes all the information about the s